# Introduction

goatools wants to read an NCBI gene2go file for building its gene dictionary... I have Ensembl IDs. I found a uniprot database that has mappings...

Merging other files didn't work.

I found on <a href="http://useast.ensembl.org/biomart/">biomart</a> a way to download Ensembl IDs to GO terms with most of the information needed to generate a gene2go file

Target columns are:

tax_id GeneID  GO_ID   Evidence        Qualifier       GO_term PubMed  Category


In [1]:
import pandas
import xopen

In [2]:
ensembl_biomart = pandas.read_csv('ensmbl_go.txt', sep='\t')

In [3]:
ensembl_biomart.columns

Index(['Gene stable ID', 'Gene stable ID version', 'GO term accession',
       'GO term name', 'GO term evidence code', 'GO domain'],
      dtype='object')

In [4]:
def ensembl_domain_to_category(x):
    return {
        'biological_process': 'Process',
        'cellular_component': 'Component',
        'molecular_function': 'Function',
    }.get(x, x)

In [16]:
ensembl_biomart['#tax_id'] = '10090'
ensembl_biomart['PubMed'] = '-'
ensembl_biomart['Category'] = ensembl_biomart['GO domain'].apply(ensembl_domain_to_category)
ensembl_biomart['Qualifier'] = '-'
ensembl_go = ensembl_biomart.rename(columns={
    'Gene stable ID version': 'GeneID',
    'GO term accession': 'GO_ID',
    'GO term evidence code': 'Evidence',
    'GO term name': 'GO_term',
})[["#tax_id", "GeneID", "GO_ID", "Evidence", "Qualifier", "GO_term", "PubMed", "Category"]].dropna(how='any', subset=['GO_ID', 'Category'])
print(ensembl_go.shape)
ensembl_go.head()

(930750, 8)


,#tax_id,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category
2,10090,ENSMUSG00000064370.1,GO:0022904,IEA,-,respiratory electron transport chain,-,Process
3,10090,ENSMUSG00000064370.1,GO:0016020,IEA,-,membrane,-,Component
4,10090,ENSMUSG00000064370.1,GO:0016021,IEA,-,integral component of membrane,-,Component
5,10090,ENSMUSG00000064370.1,GO:0046872,IEA,-,metal ion binding,-,Function
6,10090,ENSMUSG00000064370.1,GO:0032991,ISO,-,protein-containing complex,-,Component


In [17]:
filename = 'ensembl_gene2go'
#out = xopen.xopen('ensembl_gene2go.gz', 'wt')
ensembl_go.fillna('').to_csv(filename, sep='\t', index=False)
#out.close()